# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import gmaps.datasets
import ipywidgets as widgets

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Lüderitz,0,NaN,1601721127,56,-26.65,15.16,64.53,14.32
1,Sur,0,OM,1601721127,38,22.57,59.53,90.90,7.87
2,Ahipara,4,NZ,1601721107,82,-35.17,173.17,51.91,2.77
3,Pimenta Bueno,65,BR,1601721128,84,-11.67,-61.19,74.25,1.34
4,Nikolskoye,52,RU,1601721047,62,59.70,30.79,63.00,13.42
...,...,...,...,...,...,...,...,...,...
556,Yokadouma,96,CM,1601721283,68,3.52,15.05,83.82,1.74
557,Adré,0,TD,1601721283,37,13.47,22.20,90.27,6.38
558,Mumbai,75,IN,1601721056,79,19.01,72.85,87.80,11.41
559,Bandrele,24,YT,1601721284,66,-12.91,45.19,86.00,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
31,Rodolfo Sánchez Taboada,0,MX,1601721136,43,31.72,-116.57,70.50,0.96
106,Cabo San Lucas,0,MX,1601721153,77,22.89,-109.91,77.00,1.01
163,Romitan,0,UZ,1601721169,15,39.93,64.38,77.00,6.93
301,São João da Barra,0,BR,1601721043,88,-21.64,-41.05,75.20,9.17
340,Mashhad,0,IR,1601720931,15,36.30,59.61,75.20,9.17
356,Jutaí,0,BR,1601721224,87,-5.18,-68.90,73.99,0.40
366,Yangi Marg`ilon,0,UZ,1601721227,18,40.43,71.72,71.60,6.93
388,Guerrero Negro,0,MX,1601721040,68,27.98,-114.06,72.28,5.03
468,Beyneu,0,KZ,1601721233,18,45.32,55.20,72.70,6.49


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
31,Rodolfo Sánchez Taboada,MX,31.72,-116.57,
106,Cabo San Lucas,MX,22.89,-109.91,
163,Romitan,UZ,39.93,64.38,
301,São João da Barra,BR,-21.64,-41.05,
340,Mashhad,IR,36.30,59.61,
356,Jutaí,BR,-5.18,-68.90,
366,Yangi Marg`ilon,UZ,40.43,71.72,
388,Guerrero Negro,MX,27.98,-114.06,
468,Beyneu,KZ,45.32,55.20,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 31: Rodolfo Sánchez Taboada.
Missing field/result... skipping.
------------
Retrieving Results for Index 106: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Waldorf Astoria Los Cabos Pedregal.
------------
Retrieving Results for Index 163: Romitan.
Missing field/result... skipping.
------------
Retrieving Results for Index 301: São João da Barra.
Closest hotel in São João da Barra is Pousada Porto Prime.
------------
Retrieving Results for Index 340: Mashhad.
Closest hotel in Mashhad is Boshra Hotel.
------------
Retrieving Results for Index 356: Jutaí.
Missing field/result... skipping.
------------
Retrieving Results for Index 366: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is ATLAS HOTEL.
------------
Retrieving Results for Index 388: Guerrero Negro.
Closest hotel in Guerrero Negro is Casa Laguna, Bed & Breakfast.
------------
Retrieving Results for Index 468: Beyneu.
Closest hotel in Beyneu is HOTEL "AKNUR".
------------
-------End of Search--

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
31,Rodolfo Sánchez Taboada,MX,31.72,-116.57,
106,Cabo San Lucas,MX,22.89,-109.91,Waldorf Astoria Los Cabos Pedregal
163,Romitan,UZ,39.93,64.38,
301,São João da Barra,BR,-21.64,-41.05,Pousada Porto Prime
340,Mashhad,IR,36.30,59.61,Boshra Hotel
356,Jutaí,BR,-5.18,-68.90,
366,Yangi Marg`ilon,UZ,40.43,71.72,ATLAS HOTEL
388,Guerrero Negro,MX,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
468,Beyneu,KZ,45.32,55.20,"HOTEL ""AKNUR"""


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)